# Circuit learning module: Lambeq manually with SPSA and JAX

This module performs the optimization of the parametrized circuit manually compared to Lambeq's automatic QuantumTrainer class. I created this because I wanted to have more control over the optimization process and debug it better. The code is based on the workflow presented in https://github.com/CQCL/Quanthoven.

In [1]:
import warnings
import json
import os
import sys
import glob
from math import ceil
from pathlib import Path
from jax import numpy as np
from sympy import default_sort_key
import numpy
import pickle
import matplotlib.pyplot as plt

import jax
from jax import jit
from noisyopt import minimizeSPSA, minimizeCompass

from discopy.quantum import Circuit
from discopy.tensor import Tensor
from discopy.utils import loads
#from pytket.extensions.qiskit import AerBackend
#from pytket.extensions.qulacs import QulacsBackend
#from pytket.extensions.cirq import CirqStateSampleBackend
backend = None

from utils import *
#from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

warnings.filterwarnings('ignore')
this_folder = os.path.abspath(os.getcwd())
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
#os.environ["JAX_PLATFORMS"] = "cpu"

SEED = 0

# This avoids TracerArrayConversionError from jax
Tensor.np = np

rng = numpy.random.default_rng(SEED)
numpy.random.seed(SEED)

## Read circuit data

We read the circuits from the pickled files. Select if we perform binary classification or multi-class classification. Give number of qubits to create classes:
- 1 qubits -> 2^1 = 2 classes i.e. binary classification
- 2 qubits -> 2^2 = 4 classes
- ...
- 5 qubits -> 2^5 = 32 classes, etc.

In [2]:
# Select workload
#workload = "execution_time"
workload = "cardinality"

# Select workload size
#workload_size = "small"
#workload_size = "medium"
#workload_size = "large"
workload_size = "main"

classification = 2
layers = 1
single_qubit_params = 3
n_wire_count = 1

loss = multi_class_loss
acc = multi_class_acc

if classification == 1:
    loss = bin_class_loss
    acc = bin_class_acc

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//" + workload + "//" + workload_size + "//circuits//" + str(classification) + "//" + str(layers) + "_layer//" + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count) + "_n_wire_count//"

training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [3]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

## Read training and test data

In [4]:
training_data, test_data, validation_data = None, None, None
data_path = this_folder + "//data//" + workload + "//" + workload_size + "//"

with open(data_path + "training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(data_path + "test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
with open(data_path + "validation_data.json", "r") as inputfile:
    validation_data = json.load(inputfile)['validation_data']

training_data_labels, classes = create_labeled_training_classes(training_data, classification, workload)
test_data_labels = create_labeled_test_validation_classes(test_data, classes, workload)
validation_data_labels = create_labeled_test_validation_classes(validation_data, classes, workload)

## Model

In [5]:
def make_pred_fn(circuits):
    # In the case we want to use other backends. 
    # Currently does not work properly.
    if backend:
        compiled_circuits1 = backend.get_compiled_circuits([c.to_tk() for c in circuits])
        circuits = [Circuit.from_tk(c) for c in compiled_circuits1]
        
    circuit_fns = [c.lambdify(*parameters) for c in circuits]
    
    def predict(params):
        outputs = Circuit.eval(*(c(*params) for c in circuit_fns), backend = backend)
        res = []
        
        for output in outputs:
            predictions = np.abs(output.array) + 1e-9
            ratio = predictions / predictions.sum()
            res.append(ratio)
            
        return np.array(res)
    return predict

## Loss function and evaluation

In [6]:
def make_cost_fn(pred_fn, labels):
    def cost_fn(params, **kwargs):
        predictions = pred_fn(params)
        #print(predictions, labels)
        
        cost = loss(predictions, labels) #-np.sum(labels * np.log(predictions)) / len(labels)  # binary cross-entropy loss
        costs.append(cost)

        accuracy = acc(predictions, labels) #np.sum(np.round(predictions) == labels) / len(labels) / 2  # half due to double-counting
        accuracies.append(accuracy)

        return cost

    costs, accuracies = [], []
    return cost_fn, costs, accuracies

## Minimization with noisyopt

In [7]:
def initialize_parameters(old_params, old_values, new_params):
    new_values = list(numpy.array(rng.random(len(new_params))))
    old_param_dict = {}
    for p, v in zip(old_params, old_values):
        old_param_dict[p] = v
        
    parameters = sorted(set(old_params + new_params), key=default_sort_key)
    values = []
    for p in parameters:
        if p in old_param_dict:
            values.append(old_param_dict[p])
        else:
            values.append(new_values.pop())
            
    return parameters, np.array(values)

In [8]:
EPOCHS = 5000
initial_number_of_circuits = 20
syms = {}
limit = False
all_training_keys = list(training_circuits.keys())
initial_circuit_keys = all_training_keys[:initial_number_of_circuits + 1]
current_training_circuits = {}
result_file = workload + "_" + workload_size + "_noisyopt_4_" + str(classification) + "_" + str(layers) + "_" + str(single_qubit_params)

for k in initial_circuit_keys:
    current_training_circuits[k] = training_circuits[k]
    
syms = get_symbols(current_training_circuits)
parameters = sorted(syms, key=default_sort_key)
if initial_number_of_circuits > 10 and os.path.exists("points//" + result_file + ".npz"):
    with open("points//" + result_file + ".npz", "rb") as f:
        print("Loading parameters from file " + result_file)
        npzfile = np.load(f)
        init_params_spsa = npzfile['arr_0']
else:
    print("Initializing new parameters")
    init_params_spsa = np.array(rng.random(len(parameters)))
result = None
run = 0

Initializing new parameters


In [ ]:
result = None

for i, key in enumerate(all_training_keys[initial_number_of_circuits:]):
    print("Progress: ", round((i + initial_number_of_circuits)/len(all_training_keys), 3))
    
    if len(syms) == len(get_symbols(current_training_circuits)) and i > 0:
        if i != len(all_training_keys[1:]):
            current_training_circuits[key] = training_circuits[key]
            new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
            if result:
                parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)
                #continue
            else:
                syms = get_symbols(current_training_circuits)
                parameters = sorted(syms, key=default_sort_key)
                init_params_spsa = np.array(rng.random(len(parameters)))
    else:
        run += 1
    
    # Select those circuits from test and validation circuits which share the parameters with the current training circuits
    current_validation_circuits = select_circuits(current_training_circuits, validation_circuits)
    current_test_circuits = select_circuits(current_training_circuits, test_circuits)
    
    if len(current_validation_circuits) == 0 or len(current_test_circuits) == 0:
        continue
    
    # Create lists with circuits and their corresponding label
    training_circuits_l, training_data_labels_l = construct_data_and_labels(current_training_circuits, training_data_labels)
    validation_circuits_l, validation_data_labels_l = construct_data_and_labels(current_validation_circuits, validation_data_labels)
    test_circuits_l, test_data_labels_l = construct_data_and_labels(current_test_circuits, test_data_labels)
    
    # Limit the number of validation and test circuits to 20% of number of the training circuits
    if limit:
        val_test_circ_size = ceil(len(current_training_circuits))
        if len(current_validation_circuits) > val_test_circ_size:
            validation_circuits_l = validation_circuits_l[:val_test_circ_size]
            validation_data_labels_l = validation_data_labels_l[:val_test_circ_size]
        if len(current_test_circuits) > val_test_circ_size:
            test_circuits_l = test_circuits_l[:val_test_circ_size]
            test_data_labels_l = test_data_labels_l[:val_test_circ_size]
    
    stats = f"Number of training circuits: {len(training_circuits_l)}   "\
        + f"Number of validation circuits: {len(validation_circuits_l)}   "\
        + f"Number of test circuits: {len(test_circuits_l)}   "\
        + f"Number of parameters in model: {len(set([sym for circuit in training_circuits_l for sym in circuit.free_symbols]))}"
    
    with open("results//" + result_file + ".txt", "a") as f:
        f.write(stats + "\n")
    
    print(stats)
    
    optimization_interval = 20
    
    if result == None or run % optimization_interval == 0:
    
        train_pred_fn = jit(make_pred_fn(training_circuits_l))
        dev_pred_fn = jit(make_pred_fn(validation_circuits_l))
        test_pred_fn = make_pred_fn(test_circuits_l)

        train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
        dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)

        def callback_fn(xk):
            #print(xk)
            valid_loss = dev_cost_fn(xk)
            train_loss = numpy.around(min(float(train_costs[-1]), float(train_costs[-2])), 4)
            train_acc = numpy.around(min(float(train_accs[-1]), float(train_accs[-2])), 4)
            valid_acc = numpy.around(float(dev_accs[-1]), 4)
            iters = int(len(train_accs)/2)
            if iters % 200 == 0:
                info = f"Epoch: {iters}   "\
                + f"train/loss: {train_loss}   "\
                + f"valid/loss: {numpy.around(float(valid_loss), 4)}   "\
                + f"train/acc: {train_acc}   "\
                + f"valid/acc: {valid_acc}"

                with open("results//" + result_file + ".txt", "a") as f:
                    f.write(info + "\n")

                print(info, file=sys.stderr)
            return valid_loss

        #a_value = 0.0053
        #c_value = 0.0185

        # Good
        #a_value = 0.053
        #c_value = 0.00185
        
        a_value = 0.01 #*(1/run)
        c_value = 0.01 #*(1/run)
        
        a_values = [0.1, 0.01, 0.001, 0.0001]
        c_values = [1, 0.1, 0.01, 0.001, 0.0001]
        
        print(a_value, c_value)
        train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
        dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)
        
        #for a_value in a_values:
        #    for c_value in c_values:
        #print(a_value, c_value)
        result = minimizeSPSA(train_cost_fn, x0=init_params_spsa, a = a_value, c = c_value, niter=EPOCHS, callback=callback_fn)
        #result = minimizeCompass(train_cost_fn, x0=init_params_spsa, redfactor=2.0, deltainit=1.0, deltatol=0.001, feps=1e-15, errorcontrol=True, funcNinit=30, funcmultfactor=2.0, paired=True, alpha=0.05, callback=callback_fn)

        figure_path = this_folder + "//results//" + result_file + ".png"
        visualize_result_noisyopt(result, make_cost_fn, test_pred_fn, test_data_labels_l, train_costs, train_accs, dev_costs, dev_accs, figure_path, result_file)
    
    run += 1
    #EPOCHS += 100
    syms = get_symbols(current_training_circuits)
    
    # Extend for the next optimization round
    current_training_circuits[key] = training_circuits[key]
    new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
    parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)

Progress:  0.045
Number of training circuits: 20   Number of validation circuits: 82   Number of test circuits: 78   Number of parameters in model: 201
0.01 0.01


Epoch: 200   train/loss: 5.3728   valid/loss: 28.364   train/acc: 0.45   valid/acc: 0.2317
Epoch: 400   train/loss: 4.4005   valid/loss: 28.3532   train/acc: 0.55   valid/acc: 0.3171
Epoch: 600   train/loss: 3.8354   valid/loss: 31.5226   train/acc: 0.65   valid/acc: 0.3293
Epoch: 800   train/loss: 3.8226   valid/loss: 33.6538   train/acc: 0.6   valid/acc: 0.3537
Epoch: 1000   train/loss: 3.5984   valid/loss: 32.0436   train/acc: 0.65   valid/acc: 0.3415
Epoch: 1200   train/loss: 3.5049   valid/loss: 33.9556   train/acc: 0.65   valid/acc: 0.3415
Epoch: 1400   train/loss: 3.3355   valid/loss: 31.829   train/acc: 0.65   valid/acc: 0.3537
Epoch: 1600   train/loss: 3.4778   valid/loss: 32.1601   train/acc: 0.7   valid/acc: 0.3415
Epoch: 1800   train/loss: 3.3419   valid/loss: 33.4344   train/acc: 0.7   valid/acc: 0.3659
Epoch: 2000   train/loss: 3.0551   valid/loss: 33.6976   train/acc: 0.7   valid/acc: 0.3659
Epoch: 2200   train/loss: 3.0313   valid/loss: 33.1269   train/acc: 0.7   valid/

Test accuracy: 0.3333333333333333
Progress:  0.047
Number of training circuits: 21   Number of validation circuits: 82   Number of test circuits: 81   Number of parameters in model: 204
Progress:  0.049
Number of training circuits: 22   Number of validation circuits: 82   Number of test circuits: 81   Number of parameters in model: 204
Progress:  0.051
Number of training circuits: 23   Number of validation circuits: 82   Number of test circuits: 81   Number of parameters in model: 204
Progress:  0.054
Number of training circuits: 24   Number of validation circuits: 82   Number of test circuits: 81   Number of parameters in model: 204
Progress:  0.056
Number of training circuits: 25   Number of validation circuits: 82   Number of test circuits: 81   Number of parameters in model: 204
Progress:  0.058
Number of training circuits: 26   Number of validation circuits: 85   Number of test circuits: 86   Number of parameters in model: 207
Progress:  0.06
Number of training circuits: 27   Numb

Epoch: 200   train/loss: 12.498   valid/loss: 32.9327   train/acc: 0.2564   valid/acc: 0.3229
Epoch: 400   train/loss: 10.4553   valid/loss: 36.7214   train/acc: 0.4872   valid/acc: 0.2604
Epoch: 600   train/loss: 9.4694   valid/loss: 38.4374   train/acc: 0.5641   valid/acc: 0.25
Epoch: 800   train/loss: 8.9116   valid/loss: 39.3357   train/acc: 0.641   valid/acc: 0.2812
Epoch: 1000   train/loss: 8.0239   valid/loss: 41.1072   train/acc: 0.6667   valid/acc: 0.2604
Epoch: 1200   train/loss: 7.6066   valid/loss: 49.5457   train/acc: 0.6667   valid/acc: 0.2917
Epoch: 1400   train/loss: 6.9956   valid/loss: 46.4684   train/acc: 0.6923   valid/acc: 0.3125
Epoch: 1600   train/loss: 6.8683   valid/loss: 44.776   train/acc: 0.6923   valid/acc: 0.3021
Epoch: 1800   train/loss: 6.2906   valid/loss: 47.3618   train/acc: 0.7179   valid/acc: 0.3021
Epoch: 2000   train/loss: 6.2665   valid/loss: 45.9028   train/acc: 0.6667   valid/acc: 0.3125
Epoch: 2200   train/loss: 5.9706   valid/loss: 46.5284   

Test accuracy: 0.35714285714285715
Progress:  0.089
Number of training circuits: 40   Number of validation circuits: 99   Number of test circuits: 101   Number of parameters in model: 228
Progress:  0.092
Number of training circuits: 41   Number of validation circuits: 99   Number of test circuits: 101   Number of parameters in model: 228
Progress:  0.094
Number of training circuits: 42   Number of validation circuits: 99   Number of test circuits: 101   Number of parameters in model: 228
Progress:  0.096
Number of training circuits: 43   Number of validation circuits: 99   Number of test circuits: 101   Number of parameters in model: 228
Progress:  0.098
Number of training circuits: 44   Number of validation circuits: 99   Number of test circuits: 101   Number of parameters in model: 228
Progress:  0.1
Number of training circuits: 45   Number of validation circuits: 99   Number of test circuits: 101   Number of parameters in model: 228
Progress:  0.103
Number of training circuits: 46 

...

Epoch: 200   train/loss: 24.4502   valid/loss: 38.1636   train/acc: 0.359   valid/acc: 0.2816
Epoch: 400   train/loss: 21.3937   valid/loss: 35.0771   train/acc: 0.3846   valid/acc: 0.3204
Epoch: 600   train/loss: 20.4268   valid/loss: 35.1864   train/acc: 0.4872   valid/acc: 0.3495
Epoch: 800   train/loss: 19.6459   valid/loss: 39.0128   train/acc: 0.5513   valid/acc: 0.3786
Epoch: 1000   train/loss: 21.2014   valid/loss: 39.672   train/acc: 0.4487   valid/acc: 0.3883
Epoch: 1200   train/loss: 18.9696   valid/loss: 39.1991   train/acc: 0.5   valid/acc: 0.3107
Epoch: 1400   train/loss: 18.3641   valid/loss: 36.6433   train/acc: 0.5897   valid/acc: 0.3204
Epoch: 1600   train/loss: 17.745   valid/loss: 38.9303   train/acc: 0.5641   valid/acc: 0.2913
Epoch: 1800   train/loss: 17.7959   valid/loss: 38.2024   train/acc: 0.5256   valid/acc: 0.3204
Epoch: 2000   train/loss: 17.6234   valid/loss: 39.0609   train/acc: 0.5256   valid/acc: 0.3204
Epoch: 2200   train/loss: 17.3153   valid/loss: 43

Test accuracy: 0.3333333333333333
Progress:  0.179
Number of training circuits: 79   Number of validation circuits: 103   Number of test circuits: 105   Number of parameters in model: 231
Progress:  0.181
Number of training circuits: 80   Number of validation circuits: 103   Number of test circuits: 105   Number of parameters in model: 231
Progress:  0.183
Number of training circuits: 81   Number of validation circuits: 103   Number of test circuits: 105   Number of parameters in model: 231
Progress:  0.185
Number of training circuits: 82   Number of validation circuits: 103   Number of test circuits: 105   Number of parameters in model: 231
Progress:  0.188
Number of training circuits: 83   Number of validation circuits: 103   Number of test circuits: 105   Number of parameters in model: 231
Progress:  0.19
Number of training circuits: 84   Number of validation circuits: 103   Number of test circuits: 105   Number of parameters in model: 231
Progress:  0.192
Number of training circuit

Epoch: 200   train/loss: 33.8334   valid/loss: 35.7113   train/acc: 0.2474   valid/acc: 0.2427
Epoch: 400   train/loss: 33.0615   valid/loss: 35.0244   train/acc: 0.2062   valid/acc: 0.2913
Epoch: 600   train/loss: 30.9268   valid/loss: 34.6679   train/acc: 0.3093   valid/acc: 0.2913
Epoch: 800   train/loss: 29.8617   valid/loss: 33.2173   train/acc: 0.3196   valid/acc: 0.3107
Epoch: 1000   train/loss: 29.3666   valid/loss: 34.0171   train/acc: 0.4021   valid/acc: 0.3689
Epoch: 1200   train/loss: 28.8894   valid/loss: 34.2753   train/acc: 0.3918   valid/acc: 0.2913
Epoch: 1400   train/loss: 28.1648   valid/loss: 34.9231   train/acc: 0.3299   valid/acc: 0.3301
Epoch: 1600   train/loss: 28.0506   valid/loss: 34.8768   train/acc: 0.433   valid/acc: 0.3592
Epoch: 1800   train/loss: 27.1596   valid/loss: 33.8613   train/acc: 0.4742   valid/acc: 0.3981
Epoch: 2000   train/loss: 25.2079   valid/loss: 35.7808   train/acc: 0.5464   valid/acc: 0.3204
Epoch: 2200   train/loss: 24.279   valid/loss

Test accuracy: 0.4666666666666667
Progress:  0.223
Number of training circuits: 98   Number of validation circuits: 103   Number of test circuits: 105   Number of parameters in model: 231
Progress:  0.225
Number of training circuits: 99   Number of validation circuits: 103   Number of test circuits: 105   Number of parameters in model: 234
Progress:  0.228
Number of training circuits: 100   Number of validation circuits: 103   Number of test circuits: 105   Number of parameters in model: 234
Progress:  0.23
Number of training circuits: 101   Number of validation circuits: 103   Number of test circuits: 105   Number of parameters in model: 234
Progress:  0.232
Number of training circuits: 102   Number of validation circuits: 103   Number of test circuits: 105   Number of parameters in model: 234
Progress:  0.234
Number of training circuits: 103   Number of validation circuits: 103   Number of test circuits: 105   Number of parameters in model: 234
Progress:  0.237
Number of training cir

Epoch: 200   train/loss: 40.4721   valid/loss: 35.662   train/acc: 0.3675   valid/acc: 0.2913
Epoch: 400   train/loss: 39.2317   valid/loss: 35.3374   train/acc: 0.2991   valid/acc: 0.2621
Epoch: 600   train/loss: 37.2678   valid/loss: 35.2974   train/acc: 0.4188   valid/acc: 0.3592
Epoch: 800   train/loss: 37.1294   valid/loss: 35.5462   train/acc: 0.3932   valid/acc: 0.3689
Epoch: 1000   train/loss: 34.8296   valid/loss: 36.3674   train/acc: 0.4274   valid/acc: 0.3107
Epoch: 1200   train/loss: 35.4465   valid/loss: 35.0714   train/acc: 0.5299   valid/acc: 0.4563
